<a href="https://colab.research.google.com/github/pandaLabOS/firestore-migrator/blob/master/Copy_of_csvToFirestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#To Read
##Tutorials and Documentation
[Importing data into Firestore using Python Part 2: Data Types](https://brannen.dev/importing-data-into-firestore-using-python-part-2-data-types/)

[CSV to array in Firestore](https://stackoverflow.com/questions/62368238/importing-an-array-from-csv-to-firestore)

[Supported datatypes in Firestore](https://firebase.google.com/docs/firestore/data-model)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd/content/gdrive/My\Drive/Colab\ Notebooks/"Senior Project 1"

/content/gdrive/MyDrive/Colab Notebooks/Senior Project 1


#Python program to add data from csv files to Firestore
Source: 
- https://brannen.dev/importing-data-into-firestore-using-python-part-1/
- https://brannen.dev/importing-data-into-firestore-using-python-part-2-data-types/ 


In [ ]:
!pip install firebase-admin google-cloud-firestore

In [ ]:
import csv
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

In [ ]:
cred = credentials.Certificate("./credentials_senior1.json")
app = firebase_admin.initialize_app(cred, options = {})

store = firestore.client()

ValueError: ignored

In [ ]:
cred2 = credentials.Certificate("./credentials_senior1Alex.json")
app = firebase_admin.initialize_app(cred2)

store = firestore.client()

In [ ]:
file_path = "DrinksDatabase.csv"
collection_name = "products"

In [ ]:
def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]


def get_data_item(item, data_type):
	# Add other data types you want to handle here
    if data_type == 'int':
        return int(item)
    elif data_type == 'float':
        return float(item)
    elif data_type == 'string':
        return str(item)
    elif data_type == 'bool':
        return bool(item)
    elif data_type == 'array':
        array = list(map(str, item.split(" ")))
        array.pop()
        return array
    elif data_type == 'map':
        return dict(item)
    else:
        return item

#firestore map = dict

data = []
headers = []
data_types = []

with open(file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        print(row)
        if line_count == 0:
            for header in row:
                headers.append(header)
            line_count += 1
        elif line_count == 1:
            for data_type in row:
                data_types.append(data_type)
            line_count += 1
        else:
            obj = {}
            for index, item in enumerate(row):
                obj[headers[index]] = get_data_item(item, data_types[index])
            data.append(obj)
            line_count += 1
    print(f'Processed {line_count} lines.')

for batched_data in batch_data(data, 499):
    batch = store.batch()
    for data_item in batched_data:
        doc_ref = store.collection(collection_name).document(data_item.get("Product Code"))
        batch.set(doc_ref, data_item)
    batch.commit()

print('Done')

['\ufeffItemNo', 'Product Code', 'Product Name', 'Retail', 'Barcode', 'Alc.', 'Unit', 'Size', 'Subtype_TH', 'Type_TH', 'Subtype_EN', 'Type_EN', 'Occasion_List']
['int', 'string', 'string', 'int', 'string', 'float', 'string', 'int', 'string', 'string', 'string', 'string', 'array']
['1', 'SPW01-VFT', 'Villafresno Tinto', '450', '885 92732 0000 4', '0.12', 'Bottle', '750', 'ไวน์แดง', 'ไวน์', 'Red Wine', 'Wine', 'Casual Dinner ']
['2', 'SPW01-VFB', 'Villafresno Blanco', '450', '885 92732 0001 1', '0.12', 'Bottle', '750', 'ไวน์ขาว', 'ไวน์', 'White Wine', 'Wine', 'Casual Dinner ']
['3', 'SPW01-VFS', 'Villafresno Rosado', '450', '885 92732 0002 8', '0.12', 'Bottle', '750', 'ไวน์โรเซ่', 'ไวน์', 'Rose Wine', 'Wine', 'Casual Dinner ']
['4', 'SPW01-IRE', 'Irene ', '465', '885 92732 0007 3', '0.11', 'Bottle', '750', 'ไวน์สปาร์คกลิ้ง', 'ไวน์', 'Sparkling Wine', 'Wine', 'Anniversary Birthday Celebration Dinner ']
['5', 'SPW01-SLR', 'Solero Sangria ', '410', '885 92732 0003 5', '0.07', 'Bottle', '100